# Create Output Port Table in Unity Catalog

This notebook uses the datacontract.yaml to export a CREATE TABLE SQL-statement from the defined model.
Then it actually creates the table in unity catalog.

In [ ]:
def get_absolute_path(*relative_parts):
    import os
    if 'dbutils' in globals():
        # dbutils is available to the entry point when run in Databricks
        base_dir = os.path.dirname(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()) # type: ignore
        path = os.path.normpath(os.path.join(base_dir, *relative_parts))
        return path if path.startswith("/Workspace") else "/Workspace" + path
    else:
        return os.path.join(*relative_parts)


## Export Data Contract as SQL
Generate the CREATE TABLE STATEMENT

In [4]:
from datacontract.data_contract import DataContract
   
data_contract_file = get_absolute_path("..", "datacontract.yaml")
data_contract = DataContract(data_contract_file=data_contract_file)
sql_ddl = data_contract.export(export_format="sql")
print(sql_ddl)

KeyError: 'dbutils'

## Create Unity Catalog table

The Data Catalog _acme_ must exist.

In [ ]:
from helpers.spark import get_spark

# Ensure the schema exists
get_spark().sql("CREATE SCHEMA IF NOT EXISTS acme.stock_last_sales")

# Create the table in Unity Catalog
get_spark().sql(sql_ddl)